# Imagenet Evaluation Script
modified from [the evluation script by OpenAI](https://colab.research.google.com/github/openai/clip/blob/master/notebooks/Prompt_Engineering_for_ImageNet.ipynb).

In [1]:
import json

import jax
from jax import numpy as jnp

import os 
os.environ['TOKENIZERS_PARALLELISM'] = "false"

import transformers
from transformers import AutoTokenizer

import numpy as np
import torch
import torchvision
from torchvision import transforms
from torchvision.transforms import CenterCrop, ConvertImageDtype, Normalize, Resize, ColorJitter, RandomHorizontalFlip, RandomRotation, ToTensor, Lambda
from torchvision.transforms.functional import InterpolationMode
from tqdm.notebook import tqdm

!wget -q https://github.com/huggingface/transformers/raw/master/examples/research_projects/jax-projects/hybrid_clip/modeling_hybrid_clip.py
!wget -q https://github.com/huggingface/transformers/raw/master/examples/research_projects/jax-projects/hybrid_clip/configuration_hybrid_clip.py

import sys
sys.path.append('/home/raphaelp/clip-italian/hybrid_clip/')

from modeling_hybrid_clip import FlaxHybridCLIP
# from configuration_hybrid_clip import HybridCLIPConfig

In [6]:
# Global Variables

LANGUAGE = 'it'
IMAGENET_ROOT = "/home/raphaelp/imagenet_root/"


if LANGUAGE == 'en':
    TOKENIZER_NAME = "roberta-base" 
    CONFIG_FILE = '/home/raphaelp/clip-base/config.json'
    MODEL_FILE = '/home/raphaelp/clip-base/checkpoint/flax_model.msgpack'
    with open(CONFIG_FILE, 'r') as f:
        config_dict = json.load(f)
    config_dict['vision_config']['model_type'] = 'clip'
    config = HybridCLIPConfig(text_config_dict=config_dict['text_config'], vision_config_dict=config_dict['vision_config'])
    model = FlaxHybridCLIP.from_pretrained(MODEL_FILE, config=config)
elif LANGUAGE == 'it':
    TOKENIZER_NAME = "dbmdz/bert-base-italian-cased"
    model = model = FlaxHybridCLIP.from_pretrained("clip-italian/clip-italian")
    
    
config = model.config

Some weights of FlaxHybridCLIP were not initialized from the model checkpoint at clip-italian/clip-italian and are newly initialized: {('logit_scale',)}
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Loading the model

In [7]:
tokenizer = AutoTokenizer.from_pretrained(
    TOKENIZER_NAME, cache_dir=None, use_fast=True
)

In [8]:
# with open(CONFIG_FILE, 'r') as f:
#     config_dict = json.load(f)
# config_dict['vision_config']['model_type'] = 'clip'
# config = HybridCLIPConfig(text_config_dict=config_dict['text_config'], vision_config_dict=config_dict['vision_config'])
# model = FlaxHybridCLIP.from_pretrained(MODEL_FILE, config=config)

In [9]:
print("Input resolution:", config.vision_config.image_size)
print("Context length:", config.text_config.max_length)
print("Vocab size:", config.text_config.vocab_size)

Input resolution: 224
Context length: 20
Vocab size: 32102


# Preparing ImageNet labels and prompts

The following cell contains the 1,000 labels for the ImageNet dataset, followed by the text templates we'll use as "prompt engineering".

In [10]:
if LANGUAGE == 'en':
    imagenet_classes = ["tench", "goldfish", "great white shark", "tiger shark", "hammerhead shark", "electric ray", "stingray", "rooster", "hen", "ostrich", "brambling", "goldfinch", "house finch", "junco", "indigo bunting", "American robin", "bulbul", "jay", "magpie", "chickadee", "American dipper", "kite (bird of prey)", "bald eagle", "vulture", "great grey owl", "fire salamander", "smooth newt", "newt", "spotted salamander", "axolotl", "American bullfrog", "tree frog", "tailed frog", "loggerhead sea turtle", "leatherback sea turtle", "mud turtle", "terrapin", "box turtle", "banded gecko", "green iguana", "Carolina anole", "desert grassland whiptail lizard", "agama", "frilled-necked lizard", "alligator lizard", "Gila monster", "European green lizard", "chameleon", "Komodo dragon", "Nile crocodile", "American alligator", "triceratops", "worm snake", "ring-necked snake", "eastern hog-nosed snake", "smooth green snake", "kingsnake", "garter snake", "water snake", "vine snake", "night snake", "boa constrictor", "African rock python", "Indian cobra", "green mamba", "sea snake", "Saharan horned viper", "eastern diamondback rattlesnake", "sidewinder rattlesnake", "trilobite", "harvestman", "scorpion", "yellow garden spider", "barn spider", "European garden spider", "southern black widow", "tarantula", "wolf spider", "tick", "centipede", "black grouse", "ptarmigan", "ruffed grouse", "prairie grouse", "peafowl", "quail", "partridge", "african grey parrot", "macaw", "sulphur-crested cockatoo", "lorikeet", "coucal", "bee eater", "hornbill", "hummingbird", "jacamar", "toucan", "duck", "red-breasted merganser", "goose", "black swan", "tusker", "echidna", "platypus", "wallaby", "koala", "wombat", "jellyfish", "sea anemone", "brain coral", "flatworm", "nematode", "conch", "snail", "slug", "sea slug", "chiton", "chambered nautilus", "Dungeness crab", "rock crab", "fiddler crab", "red king crab", "American lobster", "spiny lobster", "crayfish", "hermit crab", "isopod", "white stork", "black stork", "spoonbill", "flamingo", "little blue heron", "great egret", "bittern bird", "crane bird", "limpkin", "common gallinule", "American coot", "bustard", "ruddy turnstone", "dunlin", "common redshank", "dowitcher", "oystercatcher", "pelican", "king penguin", "albatross", "grey whale", "killer whale", "dugong", "sea lion", "Chihuahua", "Japanese Chin", "Maltese", "Pekingese", "Shih Tzu", "King Charles Spaniel", "Papillon", "toy terrier", "Rhodesian Ridgeback", "Afghan Hound", "Basset Hound", "Beagle", "Bloodhound", "Bluetick Coonhound", "Black and Tan Coonhound", "Treeing Walker Coonhound", "English foxhound", "Redbone Coonhound", "borzoi", "Irish Wolfhound", "Italian Greyhound", "Whippet", "Ibizan Hound", "Norwegian Elkhound", "Otterhound", "Saluki", "Scottish Deerhound", "Weimaraner", "Staffordshire Bull Terrier", "American Staffordshire Terrier", "Bedlington Terrier", "Border Terrier", "Kerry Blue Terrier", "Irish Terrier", "Norfolk Terrier", "Norwich Terrier", "Yorkshire Terrier", "Wire Fox Terrier", "Lakeland Terrier", "Sealyham Terrier", "Airedale Terrier", "Cairn Terrier", "Australian Terrier", "Dandie Dinmont Terrier", "Boston Terrier", "Miniature Schnauzer", "Giant Schnauzer", "Standard Schnauzer", "Scottish Terrier", "Tibetan Terrier", "Australian Silky Terrier", "Soft-coated Wheaten Terrier", "West Highland White Terrier", "Lhasa Apso", "Flat-Coated Retriever", "Curly-coated Retriever", "Golden Retriever", "Labrador Retriever", "Chesapeake Bay Retriever", "German Shorthaired Pointer", "Vizsla", "English Setter", "Irish Setter", "Gordon Setter", "Brittany dog", "Clumber Spaniel", "English Springer Spaniel", "Welsh Springer Spaniel", "Cocker Spaniel", "Sussex Spaniel", "Irish Water Spaniel", "Kuvasz", "Schipperke", "Groenendael dog", "Malinois", "Briard", "Australian Kelpie", "Komondor", "Old English Sheepdog", "Shetland Sheepdog", "collie", "Border Collie", "Bouvier des Flandres dog", "Rottweiler", "German Shepherd Dog", "Dobermann", "Miniature Pinscher", "Greater Swiss Mountain Dog", "Bernese Mountain Dog", "Appenzeller Sennenhund", "Entlebucher Sennenhund", "Boxer", "Bullmastiff", "Tibetan Mastiff", "French Bulldog", "Great Dane", "St. Bernard", "husky", "Alaskan Malamute", "Siberian Husky", "Dalmatian", "Affenpinscher", "Basenji", "pug", "Leonberger", "Newfoundland dog", "Great Pyrenees dog", "Samoyed", "Pomeranian", "Chow Chow", "Keeshond", "brussels griffon", "Pembroke Welsh Corgi", "Cardigan Welsh Corgi", "Toy Poodle", "Miniature Poodle", "Standard Poodle", "Mexican hairless dog (xoloitzcuintli)", "grey wolf", "Alaskan tundra wolf", "red wolf or maned wolf", "coyote", "dingo", "dhole", "African wild dog", "hyena", "red fox", "kit fox", "Arctic fox", "grey fox", "tabby cat", "tiger cat", "Persian cat", "Siamese cat", "Egyptian Mau", "cougar", "lynx", "leopard", "snow leopard", "jaguar", "lion", "tiger", "cheetah", "brown bear", "American black bear", "polar bear", "sloth bear", "mongoose", "meerkat", "tiger beetle", "ladybug", "ground beetle", "longhorn beetle", "leaf beetle", "dung beetle", "rhinoceros beetle", "weevil", "fly", "bee", "ant", "grasshopper", "cricket insect", "stick insect", "cockroach", "praying mantis", "cicada", "leafhopper", "lacewing", "dragonfly", "damselfly", "red admiral butterfly", "ringlet butterfly", "monarch butterfly", "small white butterfly", "sulphur butterfly", "gossamer-winged butterfly", "starfish", "sea urchin", "sea cucumber", "cottontail rabbit", "hare", "Angora rabbit", "hamster", "porcupine", "fox squirrel", "marmot", "beaver", "guinea pig", "common sorrel horse", "zebra", "pig", "wild boar", "warthog", "hippopotamus", "ox", "water buffalo", "bison", "ram (adult male sheep)", "bighorn sheep", "Alpine ibex", "hartebeest", "impala (antelope)", "gazelle", "arabian camel", "llama", "weasel", "mink", "European polecat", "black-footed ferret", "otter", "skunk", "badger", "armadillo", "three-toed sloth", "orangutan", "gorilla", "chimpanzee", "gibbon", "siamang", "guenon", "patas monkey", "baboon", "macaque", "langur", "black-and-white colobus", "proboscis monkey", "marmoset", "white-headed capuchin", "howler monkey", "titi monkey", "Geoffroy's spider monkey", "common squirrel monkey", "ring-tailed lemur", "indri", "Asian elephant", "African bush elephant", "red panda", "giant panda", "snoek fish", "eel", "silver salmon", "rock beauty fish", "clownfish", "sturgeon", "gar fish", "lionfish", "pufferfish", "abacus", "abaya", "academic gown", "accordion", "acoustic guitar", "aircraft carrier", "airliner", "airship", "altar", "ambulance", "amphibious vehicle", "analog clock", "apiary", "apron", "trash can", "assault rifle", "backpack", "bakery", "balance beam", "balloon", "ballpoint pen", "Band-Aid", "banjo", "baluster / handrail", "barbell", "barber chair", "barbershop", "barn", "barometer", "barrel", "wheelbarrow", "baseball", "basketball", "bassinet", "bassoon", "swimming cap", "bath towel", "bathtub", "station wagon", "lighthouse", "beaker", "military hat (bearskin or shako)", "beer bottle", "beer glass", "bell tower", "baby bib", "tandem bicycle", "bikini", "ring binder", "binoculars", "birdhouse", "boathouse", "bobsleigh", "bolo tie", "poke bonnet", "bookcase", "bookstore", "bottle cap", "hunting bow", "bow tie", "brass memorial plaque", "bra", "breakwater", "breastplate", "broom", "bucket", "buckle", "bulletproof vest", "high-speed train", "butcher shop", "taxicab", "cauldron", "candle", "cannon", "canoe", "can opener", "cardigan", "car mirror", "carousel", "tool kit", "cardboard box / carton", "car wheel", "automated teller machine", "cassette", "cassette player", "castle", "catamaran", "CD player", "cello", "mobile phone", "chain", "chain-link fence", "chain mail", "chainsaw", "storage chest", "chiffonier", "bell or wind chime", "china cabinet", "Christmas stocking", "church", "movie theater", "cleaver", "cliff dwelling", "cloak", "clogs", "cocktail shaker", "coffee mug", "coffeemaker", "spiral or coil", "combination lock", "computer keyboard", "candy store", "container ship", "convertible", "corkscrew", "cornet", "cowboy boot", "cowboy hat", "cradle", "construction crane", "crash helmet", "crate", "infant bed", "Crock Pot", "croquet ball", "crutch", "cuirass", "dam", "desk", "desktop computer", "rotary dial telephone", "diaper", "digital clock", "digital watch", "dining table", "dishcloth", "dishwasher", "disc brake", "dock", "dog sled", "dome", "doormat", "drilling rig", "drum", "drumstick", "dumbbell", "Dutch oven", "electric fan", "electric guitar", "electric locomotive", "entertainment center", "envelope", "espresso machine", "face powder", "feather boa", "filing cabinet", "fireboat", "fire truck", "fire screen", "flagpole", "flute", "folding chair", "football helmet", "forklift", "fountain", "fountain pen", "four-poster bed", "freight car", "French horn", "frying pan", "fur coat", "garbage truck", "gas mask or respirator", "gas pump", "goblet", "go-kart", "golf ball", "golf cart", "gondola", "gong", "gown", "grand piano", "greenhouse", "radiator grille", "grocery store", "guillotine", "hair clip", "hair spray", "half-track", "hammer", "hamper", "hair dryer", "hand-held computer", "handkerchief", "hard disk drive", "harmonica", "harp", "combine harvester", "hatchet", "holster", "home theater", "honeycomb", "hook", "hoop skirt", "gymnastic horizontal bar", "horse-drawn vehicle", "hourglass", "iPod", "clothes iron", "carved pumpkin", "jeans", "jeep", "T-shirt", "jigsaw puzzle", "rickshaw", "joystick", "kimono", "knee pad", "knot", "lab coat", "ladle", "lampshade", "laptop computer", "lawn mower", "lens cap", "letter opener", "library", "lifeboat", "lighter", "limousine", "ocean liner", "lipstick", "slip-on shoe", "lotion", "music speaker", "loupe magnifying glass", "sawmill", "magnetic compass", "messenger bag", "mailbox", "tights", "one-piece bathing suit", "manhole cover", "maraca", "marimba", "mask", "matchstick", "maypole", "maze", "measuring cup", "medicine cabinet", "megalith", "microphone", "microwave oven", "military uniform", "milk can", "minibus", "miniskirt", "minivan", "missile", "mitten", "mixing bowl", "mobile home", "ford model t", "modem", "monastery", "monitor", "moped", "mortar and pestle", "graduation cap", "mosque", "mosquito net", "vespa", "mountain bike", "tent", "computer mouse", "mousetrap", "moving van", "muzzle", "metal nail", "neck brace", "necklace", "baby pacifier", "notebook computer", "obelisk", "oboe", "ocarina", "odometer", "oil filter", "pipe organ", "oscilloscope", "overskirt", "bullock cart", "oxygen mask", "product packet / packaging", "paddle", "paddle wheel", "padlock", "paintbrush", "pajamas", "palace", "pan flute", "paper towel", "parachute", "parallel bars", "park bench", "parking meter", "railroad car", "patio", "payphone", "pedestal", "pencil case", "pencil sharpener", "perfume", "Petri dish", "photocopier", "plectrum", "Pickelhaube", "picket fence", "pickup truck", "pier", "piggy bank", "pill bottle", "pillow", "ping-pong ball", "pinwheel", "pirate ship", "drink pitcher", "block plane", "planetarium", "plastic bag", "plate rack", "farm plow", "plunger", "Polaroid camera", "pole", "police van", "poncho", "pool table", "soda bottle", "plant pot", "potter's wheel", "power drill", "prayer rug", "printer", "prison", "missile", "projector", "hockey puck", "punching bag", "purse", "quill", "quilt", "race car", "racket", "radiator", "radio", "radio telescope", "rain barrel", "recreational vehicle", "fishing casting reel", "reflex camera", "refrigerator", "remote control", "restaurant", "revolver", "rifle", "rocking chair", "rotisserie", "eraser", "rugby ball", "ruler measuring stick", "sneaker", "safe", "safety pin", "salt shaker", "sandal", "sarong", "saxophone", "scabbard", "weighing scale", "school bus", "schooner", "scoreboard", "CRT monitor", "screw", "screwdriver", "seat belt", "sewing machine", "shield", "shoe store", "shoji screen / room divider", "shopping basket", "shopping cart", "shovel", "shower cap", "shower curtain", "ski", "balaclava ski mask", "sleeping bag", "slide rule", "sliding door", "slot machine", "snorkel", "snowmobile", "snowplow", "soap dispenser", "soccer ball", "sock", "solar thermal collector", "sombrero", "soup bowl", "keyboard space bar", "space heater", "space shuttle", "spatula", "motorboat", "spider web", "spindle", "sports car", "spotlight", "stage", "steam locomotive", "through arch bridge", "steel drum", "stethoscope", "scarf", "stone wall", "stopwatch", "stove", "strainer", "tram", "stretcher", "couch", "stupa", "submarine", "suit", "sundial", "sunglasses", "sunglasses", "sunscreen", "suspension bridge", "mop", "sweatshirt", "swim trunks / shorts", "swing", "electrical switch", "syringe", "table lamp", "tank", "tape player", "teapot", "teddy bear", "television", "tennis ball", "thatched roof", "front curtain", "thimble", "threshing machine", "throne", "tile roof", "toaster", "tobacco shop", "toilet seat", "torch", "totem pole", "tow truck", "toy store", "tractor", "semi-trailer truck", "tray", "trench coat", "tricycle", "trimaran", "tripod", "triumphal arch", "trolleybus", "trombone", "hot tub", "turnstile", "typewriter keyboard", "umbrella", "unicycle", "upright piano", "vacuum cleaner", "vase", "vaulted or arched ceiling", "velvet fabric", "vending machine", "vestment", "viaduct", "violin", "volleyball", "waffle iron", "wall clock", "wallet", "wardrobe", "military aircraft", "sink", "washing machine", "water bottle", "water jug", "water tower", "whiskey jug", "whistle", "hair wig", "window screen", "window shade", "Windsor tie", "wine bottle", "airplane wing", "wok", "wooden spoon", "wool", "split-rail fence", "shipwreck", "sailboat", "yurt", "website", "comic book", "crossword", "traffic or street sign", "traffic light", "dust jacket", "menu", "plate", "guacamole", "consomme", "hot pot", "trifle", "ice cream", "popsicle", "baguette", "bagel", "pretzel", "cheeseburger", "hot dog", "mashed potatoes", "cabbage", "broccoli", "cauliflower", "zucchini", "spaghetti squash", "acorn squash", "butternut squash", "cucumber", "artichoke", "bell pepper", "cardoon", "mushroom", "Granny Smith apple", "strawberry", "orange", "lemon", "fig", "pineapple", "banana", "jackfruit", "cherimoya (custard apple)", "pomegranate", "hay", "carbonara", "chocolate syrup", "dough", "meatloaf", "pizza", "pot pie", "burrito", "red wine", "espresso", "tea cup", "eggnog", "mountain", "bubble", "cliff", "coral reef", "geyser", "lakeshore", "promontory", "sandbar", "beach", "valley", "volcano", "baseball player", "bridegroom", "scuba diver", "rapeseed", "daisy", "yellow lady's slipper", "corn", "acorn", "rose hip", "horse chestnut seed", "coral fungus", "agaric", "gyromitra", "stinkhorn mushroom", "earth star fungus", "hen of the woods mushroom", "bolete", "corn cob", "toilet paper"]
elif LANGUAGE == 'it':
    imagenet_classes = ["tinca", "pesce rosso", "grande squalo bianco", "Squalo tigre", "Squalo martello", "raggio elettrico", "pastinaca", "gallo", "gallina", "struzzo", "rovo", "cardellino", "fringuello di casa", "giunco", "Zigolo indaco", "pettirosso americano", "bulbul", "ghiandaia", "gazza", "cincillà", "merlo acquaiolo americano", "Nibbio (uccello rapace)", "aquila calva", "avvoltoio", "Gufo grigio maggiore", "Salamandra pezzata", "tritone liscio", "tritone", "salamandra pezzata", "axolotl", "rana toro americana", "raganella", "Rana codina", "Tartaruga di mare loggerhead", "tartaruga di cuoio", "tartaruga fangosa", "tartaruga", "tartaruga scatola", "geco fasciato", "iguana verde", "Anolo della Carolina", "lucertola a coda di rondine del deserto", "agama", "lucertola dal collo a frange", "lucertola alligatore", "Mostro di Gila", "lucertola verde europea", "camaleonte", "Drago di Komodo", "coccodrillo del Nilo", "Alligatore americano", "triceratopo", "serpente verme", "serpente dal collo ad anelli", "serpente dal naso di maiale orientale", "serpente verde liscio", "serpente reale", "serpente giarrettiera", "serpente d\'acqua", "serpente delle vigne", "serpente notturno", "boa constrictor", "Pitone africano delle rocce", "cobra indiano", "mamba verde", "serpente di mare", "vipera cornuta del Sahara", "serpente a sonagli orientale", "serpente a sonagli laterale", "trilobite", "Uomo del raccolto", "scorpione", "ragno giallo del giardino", "ragno del granaio", "Ragno europeo del giardino", "vedova nera del sud", "tarantola", "ragno lupo", "zecca", "millepiedi", "fagiano di monte", "pernice bianca", "gallo cedrone", "fagiano di monte", "pavone", "quaglia", "pernice", "pappagallo grigio africano", "ara", "cacatua dalla cresta sulfurea", "lorichetto", "coucal", "mangiatore di api", "bucerotide", "colibrì", "jacamar", "tucano", "anatra", "merganser dal petto rosso", "oca", "cigno nero", "tusker", "echidna", "ornitorinco", "wallaby", "koala", "vombato", "medusa", "anemone di mare", "cervello di corallo", "verme piatto", "nematode", "conchiglia", "lumaca", "lumaca", "lumaca di mare", "chitone", "Nautilus a camera", "Granchio di Dungeness", "granchio di roccia", "granchio violinista", "granchio reale rosso", "aragosta americana", "aragosta spinosa", "gambero di fiume", "granchio eremita", "isopode", "cicogna bianca", "cicogna nera", "spatola", "fenicottero", "airone blu piccolo", "airone bianco", "Tarabuso", "uccello gru", "mignatta", "Gallinelle comuni", "Folaga americana", "otarda", "voltafaccia rubicondo", "dunlin", "pettegola comune", "fraticello", "beccaccia di mare", "Pellicano", "Pinguino reale", "albatros", "balena grigia", "balena assassina", "dugongo", "leone marino", "Chihuahua", "Mento giapponese", "Maltese", "Pechinese", "Shih Tzu", "King Charles Spaniel", "Papillon", "Terrier giocattolo", "Rhodesian Ridgeback", "Levriero afgano", "Basset Hound", "Beagle", "Bloodhound", "Bluetick Coonhound", "Black and Tan Coonhound", "Treeing Walker Coonhound", "Foxhound inglese", "Redbone Coonhound", "borzoi", "Irish Wolfhound", "Levriero italiano", "Whippet", "Segugio Ibiziano", "Levriero norvegese", "Otterhound", "Saluki", "Deerhound scozzese", "Weimaraner", "Staffordshire Bull Terrier", "American Staffordshire Terrier", "Bedlington Terrier", "Border Terrier", "Kerry Blue Terrier", "Terrier Irlandese", "Norfolk Terrier", "Norwich Terrier", "Yorkshire Terrier", "Wire Fox Terrier", "Lakeland Terrier", "Sealyham Terrier", "Airedale Terrier", "Cairn Terrier", "Terrier Australiano", "Dandie Dinmont Terrier", "Boston Terrier", "Schnauzer in miniatura", "Schnauzer Gigante", "Schnauzer Standard", "Terrier Scozzese", "Tibetan Terrier", "Australian Silky Terrier", "Terrier Wheaten a pelo morbido", "West Highland White Terrier", "Lhasa Apso", "Flat-Coated Retriever", "Curly-coated Retriever", "Golden Retriever", "Labrador Retriever", "Chesapeake Bay Retriever", "Pointer tedesco a pelo corto", "Vizsla", "Setter inglese", "Setter irlandese", "Gordon Setter", "Cane della Bretagna", "Clumber Spaniel", "Springer Spaniel inglese", "Springer Spaniel gallese", "Cocker Spaniel", "Sussex Spaniel", "Irish Water Spaniel", "Kuvasz", "Schipperke", "Cane di Groenendael", "Malinois", "Briard", "Kelpie australiano", "Komondor", "Vecchio cane da pastore inglese", "Cane da pastore delle Shetland", "Collie", "Border Collie", "Cane Bouvier des Flandres", "Rottweiler", "Cane da pastore tedesco", "Dobermann", "Pinscher in miniatura", "Cane da montagna svizzero maggiore", "Cane da montagna bernese", "Appenzeller Sennenhund", "Entlebucher Sennenhund", "Boxer", "Bullmastiff", "Mastino tibetano", "Bulldog francese", "Alano", "San Bernardo", "husky", "Alaskan Malamute", "Husky siberiano", "Dalmata", "Affenpinscher", "Basenji", "Carlino", "Leonberger", "Cane di Terranova", "Cane dei Grandi Pirenei", "Samoiedo", "Pomerania", "Chow Chow", "Keeshond", "Griffone di Bruxelles", "Pembroke Welsh Corgi", "Cardigan Welsh Corgi", "Barboncino giocattolo", "Barboncino in miniatura", "Barboncino standard", "Cane messicano senza pelo (xoloitzcuintli)", "lupo grigio", "Lupo della tundra dell\'Alaska", "lupo rosso o lupo dalla criniera", "coyote", "dingo", "dhole", "Cane selvatico africano", "iena", "volpe rossa", "volpe rossa", "Volpe artica", "volpe grigia", "gatto soriano", "gatto tigrato", "Gatto persiano", "Gatto siamese", "Mau egiziano", "coguaro", "lince", "leopardo", "leopardo delle nevi", "giaguaro", "leone", "tigre", "ghepardo", "orso bruno", "Orso nero americano", "orso polare", "orso bradipo", "mangusta", "suricato", "scarabeo tigre", "coccinella", "scarabeo di terra", "coleottero delle corna", "scarabeo delle foglie", "scarabeo stercorario", "scarabeo rinoceronte", "tonchio", "mosca", "ape", "formica", "cavalletta", "insetto grillo", "insetto stecco", "scarafaggio", "mantide religiosa", "cicala", "cicala di mare", "pizzo", "libellula", "damigella d\'acqua", "farfalla ammiraglio rosso", "Farfalla ad anelli", "farfalla monarca", "piccola farfalla bianca", "farfalla zolfo", "farfalla dalle ali di gossamer", "stella marina", "riccio di mare", "Cetriolo di mare", "Coniglio dalla coda di cotone", "lepre", "Coniglio d\'angora", "criceto", "Istrice", "Scoiattolo volpe", "marmotta", "castoro", "Porcellino d\'India", "cavallo sauro comune", "zebra", "Maiale", "Cinghiale", "facocero", "ippopotamo", "bue", "bufalo d\'acqua", "bisonte", "ariete (pecora maschio adulto)", "pecora bighorn", "Stambecco alpino", "alcefalo", "impala (antilope)", "gazzella", "Cammello arabo", "lama", "donnola", "visone", "Puzzola europea", "furetto dai piedi neri", "lontra", "puzzola", "tasso", "armadillo", "Bradipo a tre dita", "orango", "gorilla", "scimpanzé", "gibbone", "siamang", "cercopiteco", "scimmia patas", "babbuino", "macaco", "langur", "colobo bianco e nero", "scimmia proboscide", "uistitì", "cappuccino dalla testa bianca", "scimmia urlatrice", "scimmia titi", "Scimmia ragno di Geoffroy", "scimmia scoiattolo comune", "lemure dalla coda ad anelli", "indri", "Elefante asiatico", "Elefante africano della savana", "panda rosso", "panda gigante", "pesce snoek", "anguilla", "salmone argentato", "pesce bellezza di roccia", "pesce pagliaccio", "storione", "pesce aguglia", "pesce leone", "pesce palla", "abaco", "abaya", "abito accademico", "fisarmonica", "chitarra acustica", "Portaerei", "aereo di linea", "dirigibile", "altare", "ambulanza", "veicolo anfibio", "orologio analogico", "apiario", "grembiule", "bidone della spazzatura", "fucile d\'assalto", "zaino", "panificio", "Trave di equilibrio", "Palloncino", "penna a sfera", "Cerotto", "banjo", "balaustra / corrimano", "bilanciere", "barber chair", "barbershop", "fienile", "barometro", "barile", "carriola", "baseball", "basket", "bassetto", "fagotto", "cuffia", "telo da bagno", "vasca da bagno", "station wagon", "faro", "bicchiere", "cappello militare (pelle d\'orso o shako)", "bottiglia di birra", "bicchiere di birra", "campanile", "bavaglino per bambini", "bicicletta tandem", "bikini", "raccoglitore ad anelli", "binocolo", "casetta per uccelli", "rimessa per barche", "bob", "cravatta bolo", "poke bonnet", "libreria", "libreria", "tappo di bottiglia", "arco da caccia", "papillon", "targa commemorativa in ottone", "reggiseno", "frangiflutti", "pettorale", "scopa", "secchio", "fibbia", "giubbotto antiproiettile", "treno ad alta velocità", "macelleria", "taxi", "calderone", "candela", "cannone", "canoa", "apriscatole", "cardigan", "auto specchio", "carosello", "kit di attrezzi", "scatola di cartone / cartone", "ruota dell\'auto", "sportello automatico", "cassetta", "lettore di cassette", "castello", "catamarano", "CD player", "violoncello", "telefono cellulare", "catena", "chain-link fence", "chain mail", "motosega", "cassapanca" , "chiffonier", "campana o campana a vento", "armadio delle porcellane", "Calza di Natale", "chiesa", "cinema", "mannaia", "dimora sulla scogliera", "mantello", "zoccoli", "shaker per cocktail", "tazza da caffè", "coffeemaker", "spirale o bobina", "serratura a combinazione", "tastiera del computer", "negozio di caramelle", "nave container", "convertibile", "cavatappi", "cornetto", "cowboy boot", "cowboy hat", "culla", "gru da costruzione", "casco di protezione", "cassa", "letto per bambini", "Crock Pot", "palla da croquet", "Stampella", "corazza", "diga", "scrivania", "desktop computer", "telefono a disco", "pannolino", "orologio digitale", "orologio digitale", "tavolo da pranzo", "strofinaccio", "lavastoviglie", "freno a disco", "bacino", "slitta per cani", "cupola", "zerbino", "impianto di perforazione", "tamburo", "bacchetta", "manubrio", "forno olandese", "ventilatore elettrico", "chitarra elettrica", "locomotiva elettrica", "centro d\'intrattenimento", "busta", "macchina per caffè espresso", "polvere per il viso", "boa di piume", "schedario", "barca dei pompieri", "camion dei pompieri", "paravento", "pennone", "flauto", "sedia pieghevole", "casco da calcio", "carrello elevatore", "fontana", "penna stilografica", "letto a baldacchino", "vagone merci", "corno francese", "padella", "cappotto di pelliccia", "camion della spazzatura", "Maschera antigas o respiratore", "pompa di benzina", "calice", "go-kart", "pallina da golf", "golf cart", "gondola", "gong", "camice", "grand piano", "serra", "radiatore a griglia", "negozio di alimentari", "ghigliottina", "molletta per capelli", "lacca per capelli", "mezzo binario", "martello", "cestino", "asciugacapelli", "computer portatile", "fazzoletto", "hard disk drive", "armonica", "arpa", "mietitrebbia", "ascia di guerra", "fondina", "home theater", "honeycomb", "gancio", "gonna a cerchio", "barra orizzontale da ginnastica", "veicolo trainato da cavalli", "clessidra", "iPod", "ferro da stiro", "zucca intagliata", "jeans", "jeep", "maglietta", "puzzle", "Risciò", "joystick", "kimono", "ginocchiera", "nodo", "camice da laboratorio", "mestolo", "paralume", "computer portatile", "tosaerba", "tappo della lente", "tagliacarte", "biblioteca", "scialuppa di salvataggio", "accendino", "limousine", "transatlantico", "Rossetto", "scarpa da ginnastica", "lozione", "altoparlante per musica", "lente d\'ingrandimento", "segheria", "bussola magnetica", "borsa del messaggero", "cassetta della posta", "collant", "costume da bagno intero", "tombino", "maraca", "marimba", "maschera", "fiammifero", "maypole", "labirinto", "misurino", "armadietto dei medicinali", "megalite", "microfono", "forno a microonde", "uniforme militare", "barattolo del latte", "minibus", "minigonna", "minivan", "missile", "guanto", "ciotola di miscelazione", "casa mobile", "ford model t", "modem", "monastero", "monitor", "motorino", "mortaio e pestello", "graduation cap", "moschea", "zanzariera", "vespa", "mountain bike", "tenda", "mouse per computer", "trappola per topi", "furgone in movimento", "museruola", "chiodo di metallo", "collare", "collana", "Ciuccio per bambini", "computer portatile", "obelisco", "oboe", "ocarina", "contachilometri", "filtro dell\'olio", "organo a canne", "oscilloscopio", "sopragonna", "carro di buoi", "maschera d\'ossigeno", "pacchetto / imballaggio del prodotto", "paletta", "ruota a pale", "Lucchetto", "pennello", "pigiama", "palazzo", "flauto di pan", "tovagliolo di carta", "paracadute", "barre parallele", "panchina", "parchimetro", "vagone ferroviario", "patio", "telefono pubblico", "piedistallo", "astuccio per matite", "temperamatite", "profumo", "Piastra di Petri", "fotocopiatrice", "plettro", "Pickelhaube", "recinzione", "pickup", "molo", "salvadanaio", "Bottiglia di pillole", "Cuscino", "palla da ping-pong", "girandola", "nave dei pirati", "brocca per bevande", "piano a blocchi", "planetario", "borsa di plastica", "portapiatti", "aratro agricolo", "stantuffo", "Macchina fotografica Polaroid", "palo", "Furgone della polizia", "Poncho", "tavolo da biliardo", "Bottiglia di soda", "Vaso per piante", "ruota da vasaio", "trapano elettrico", "tappeto da preghiera", "stampante", "prigione", "missile", "proiettore", "disco da hockey", "sacco da boxe", "borsa", "penna d\'oca", "trapunta", "auto da corsa", "racchetta", "radiatore", "radio", "radiotelescopio", "barile per la pioggia", "veicolo ricreativo", "Mulinello da pesca", "macchina fotografica reflex", "frigorifero", "telecomando", "ristorante", "revolver", "fucile", "sedia a dondolo", "girarrosto", "gomma da cancellare", "palla da rugby", "righello, bastone per misurare", "scarpa da ginnastica", "cassaforte", "spilla da balia", "saliera", "sandalo", "sarong", "sassofono", "Fodero", "bilancia", "scuolabus", "goletta", "tabellone segnapunti", "Monitor CRT", "Vite", "cacciavite", "cintura di sicurezza", "macchina da cucire", "scudo", "negozio di scarpe", "schermo shoji / divisorio per stanze", "cestino della spesa", "carrello della spesa", "pala", "cuffia da doccia", "Tenda da doccia", "sci", "passamontagna", "sacco a pelo", "regolo calcolatore", "porta scorrevole", "slot machine", "boccaglio", "motoslitta", "spazzaneve", "distributore di sapone", "pallone da calcio", "calzino", "collettore solare termico", "sombrero", "ciotola per zuppa", "barra spaziatrice della tastiera", "riscaldatore spaziale", "navetta spaziale", "spatola", "motoscafo", "ragnatela", "fuso", "auto sportiva", "riflettore", "palcoscenico", "locomotiva a vapore", "ponte ad arco passante", "tamburo in acciaio", "stetoscopio", "sciarpa", "muro di pietra", "cronometro", "stufa", "colino", "tram", "barella", "divano", "stupa", "sottomarino", "tuta", "meridiana", "sunglasses (occhiali da sole)", "occhiali da sole", "crema solare", "ponte sospeso", "mocio", "felpa", "costume da bagno / pantaloncini", "Altalena", "interruttore elettrico", "siringa", "lampada da tavolo", "Serbatoio", "mangianastri", "teiera", "orsacchiotto", "televisione", "palla da tennis", "tetto di paglia", "tenda frontale", "ditale", "macchina per la trebbiatura", "trono", "tetto di tegole", "tostapane", "tabaccheria", "tavoletta del water", "torcia", "totem", "carro attrezzi", "negozio di giocattoli", "trattore", "camion semirimorchio", "vassoio", "trench coat", "triciclo", "trimarano", "treppiede", "arco di trionfo", "trolleybus", "trombone", "vasca idromassaggio", "tornello", "tastiera della macchina da scrivere", "ombrello", "monociclo", "pianoforte verticale", "aspirapolvere", "vaso", "soffitto a volta o ad arco", "tessuto di velluto", "distributore automatico", "paramento", "viadotto", "violino", "pallavolo", "Ferro da stiro", "orologio da parete", "portafoglio", "armadio", "aereo militare", "Lavello", "lavatrice", "Bottiglia d\'acqua", "brocca d\'acqua", "torre dell\'acqua", "brocca di whisky", "Fischietto", "parrucca per capelli", "schermo della finestra", "Tenda della finestra", "cravatta Windsor", "Bottiglia di vino", "ala d\'aereo", "wok", "cucchiaio di legno", "Lana", "Recinto con travi di legno", "naufragio", "barca a vela", "yurta", "sito web", "fumetto", "cruciverba", "segnale stradale o di traffico", "semaforo", "giacca di polvere", "menu", "piatto", "guacamole", "consomme\'", "hot pot", "trifle", "gelato", "ghiacciolo", "baguette", "bagel", "pretzel", "cheeseburger", "hot dog", "purè di patate", "cavolo", "broccoli", "cavolfiore", "zucchina", "spaghetti alla zucca", "zucca a ghianda", "zucca butternut", "cetriolo", "carciofo", "peperone", "cardo", "Fungo", "Mela Granny Smith", "fragola", "arancia", "limone", "Fico", "ananas", "banana", "jackfruit", "anona (mela custard)", "melograno", "fieno", "carbonara", "sciroppo di cioccolato", "pasta", "polpettone", "pizza", "torta salata", "burrito", "vino rosso", "espresso", "tazza di tè", "zabaione", "montagna", "bolla", "scogliera", "barriera corallina", "geyser", "riva del lago", "promontorio", "barra di sabbia", "spiaggia", "valle", "vulcano", "giocatore di baseball", "sposo", "subacqueo", "colza", "margherita", "pantofola gialla della signora", "mais", "ghianda", "rosa canina", "Seme di ippocastano", "fungo del corallo", "agarico", "gyromitra", "fungo del corno di rovo", "fungo stella di terra", "fungo gallina dei boschi", "boleto", "pannocchia di mais", "carta igienica"]

In [11]:
if LANGUAGE == 'en':
    imagenet_templates = [
        'a bad photo of a {}.',
        'a photo of many {}.',
        'a sculpture of a {}.',
        'a photo of the hard to see {}.',
        'a low resolution photo of the {}.',
        'a rendering of a {}.',
        'graffiti of a {}.',
        'a bad photo of the {}.',
        'a cropped photo of the {}.',
        'a tattoo of a {}.',
        'the embroidered {}.',
        'a photo of a hard to see {}.',
        'a bright photo of a {}.',
        'a photo of a clean {}.',
        'a photo of a dirty {}.',
        'a dark photo of the {}.',
        'a drawing of a {}.',
        'a photo of my {}.',
        'the plastic {}.',
        'a photo of the cool {}.',
        'a close-up photo of a {}.',
        'a black and white photo of the {}.',
        'a painting of the {}.',
        'a painting of a {}.',
        'a pixelated photo of the {}.',
        'a sculpture of the {}.',
        'a bright photo of the {}.',
        'a cropped photo of a {}.',
        'a plastic {}.',
        'a photo of the dirty {}.',
        'a jpeg corrupted photo of a {}.',
        'a blurry photo of the {}.',
        'a photo of the {}.',
        'a good photo of the {}.',
        'a rendering of the {}.',
        'a {} in a video game.',
        'a photo of one {}.',
        'a doodle of a {}.',
        'a close-up photo of the {}.',
        'a photo of a {}.',
        'the origami {}.',
        'the {} in a video game.',
        'a sketch of a {}.',
        'a doodle of the {}.',
        'a origami {}.',
        'a low resolution photo of a {}.',
        'the toy {}.',
        'a rendition of the {}.',
        'a photo of the clean {}.',
        'a photo of a large {}.',
        'a rendition of a {}.',
        'a photo of a nice {}.',
        'a photo of a weird {}.',
        'a blurry photo of a {}.',
        'a cartoon {}.',
        'art of a {}.',
        'a sketch of the {}.',
        'a embroidered {}.',
        'a pixelated photo of a {}.',
        'itap of the {}.',
        'a jpeg corrupted photo of the {}.',
        'a good photo of a {}.',
        'a plushie {}.',
        'a photo of the nice {}.',
        'a photo of the small {}.',
        'a photo of the weird {}.',
        'the cartoon {}.',
        'art of the {}.',
        'a drawing of the {}.',
        'a photo of the large {}.',
        'a black and white photo of a {}.',
        'the plushie {}.',
        'a dark photo of a {}.',
        'itap of a {}.',
        'graffiti of the {}.',
        'a toy {}.',
        'itap of my {}.',
        'a photo of a cool {}.',
        'a photo of a small {}.',
        'a tattoo of the {}.',
    ]
elif LANGUAGE == 'it':
    imagenet_templates = [
        "una brutta foto di {}",
        "una scultura di {}",
        "una foto di {} difficilmente visibile",
        "una foto a bassa risoluzione di {}",
        "un rendering di {}",
        "graffiti di {}",
        "una pessima foto di {}",
        "una foto ritagliata di {}",
        "un tatuaggio di {}",
        "{} ricamato",
        "{} ricamata",
        "una foto luminosa di {}",
        "una foto di {} pulito",
        "una foto di {} pulita",
        "una foto di {} sporco",
        "una foto di {} sporca",
        "una foto di {} carino",
        "una foto di {} carina",
        "una foto di {} strano",
        "una foto di {} strana",
        "una foto di {} piccolo",
        "una foto di {} piccola",
        "una foto di {} largo",
        "una foto di {} larga",
        "una foto di {} grande",
        "una foto scura di {}",
        "un disegno di {}",
        "{} di plastica",
        "una foto del {} bella",
        "una foto ravvicinata di {}",
        "una foto in bianco e nero di {}",
        "un dipinto di {}",
        "una foto sgranata di {}",
        "una foto ritagliata di {}",
        "una foto sfocata di {}",
        "una buona foto di {}",
        "una riproduzione di {}",
        "un rendering di {}",
        "{} in un video gioco",
        "uno scarabocchio di {}",
        "un origami di {}",
        "uno sketch di {}",
        "una bozza di {}",
        "una foto a bassa risoluzione di {}",
        "un giocattolo di {}",
        "una resa di {}",
        "{} come cartone animato",
        "un'opera di {}",
        "un peluche di {}",
    ]

print(f"{len(imagenet_classes)} classes, {len(imagenet_templates)} templates")

1000 classes, 49 templates


# Set up Validation Set

In [12]:
val_preprocess = transforms.Compose([
    Resize([config.vision_config.image_size], interpolation=InterpolationMode.BICUBIC),
    CenterCrop(config.vision_config.image_size),
    ToTensor(),
    Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711)),
])

In [13]:
images = torchvision.datasets.ImageNet(IMAGENET_ROOT, split='val', transform=val_preprocess)
loader = torch.utils.data.DataLoader(
    images,
    batch_size=64,
    shuffle=False,
    num_workers=32,
    persistent_workers=True,
    drop_last=False
)

# Creating zero-shot classifier weights

In [14]:
def zeroshot_classifier(classnames, templates):
    zeroshot_weights = []
    for classname in tqdm(classnames):
        texts = [template.format(classname) for template in templates] #format with class
        inputs = tokenizer(texts, max_length=72, padding="max_length", return_tensors="np")
        class_embeddings = model.get_text_features(inputs['input_ids'], inputs['attention_mask'])#embed with text encoder
        class_embeddings /= jnp.linalg.norm(class_embeddings, axis=-1, keepdims=True)
        class_embedding = jnp.mean(class_embeddings, axis=0)
        class_embedding /= jnp.linalg.norm(class_embedding)
        zeroshot_weights.append(class_embedding)
    zeroshot_weights = jnp.stack(zeroshot_weights, axis=1)
    return zeroshot_weights


zeroshot_weights = zeroshot_classifier(imagenet_classes, imagenet_templates)

# Zero-shot prediction

In [15]:
def accuracy(output, target, topk=(1,)):
    # pred = output.topk(max(topk), 1, True, True)[1].t()
    pred = np.argsort(output, axis=1)[:,-max(topk):]
    correct = jnp.equal(pred, jnp.expand_dims(target, axis=-1))
    return [float(jnp.sum(jnp.reshape(correct[:k], -1), axis=0, keepdims=True)) for k in topk]

In [16]:
top_ns = [1, 5, 10, 100]
acc_counters = [0. for _ in top_ns]
n = 0.

for i, (images, target) in enumerate(tqdm(loader)):
    images = images.permute(0, 2, 3, 1).numpy()
    target = target.numpy()
    # predict
    image_features = model.get_image_features(images,)
    image_features /= jnp.linalg.norm(image_features, axis=-1, keepdims=True)
    logits = 100. * image_features @ zeroshot_weights

    # measure accuracy
    accs = accuracy(logits, target, topk=top_ns)
    for j in range(len(top_ns)):
        acc_counters[j] += accs[j]
    n += images.shape[0]

tops = {f'top{top_ns[i]}': acc_counters[i] / n * 100 for i in range(len(top_ns))}

print(tops)

/home/raphaelp/jax/lib/python3.8/site-packages/jax/_src/numpy/lax_numpy.py:4615: UserWarning: 'kind' argument to argsort is ignored.
  warnings.warn("'kind' argument to argsort is ignored.")



{'top1': 0.158, 'top5': 0.762, 'top10': 1.5859999999999999, 'top100': 9.372}


In [17]:
# for c in imagenet_classes:
#     print(f'"{c}"')

In [ ]:
'''["tinca"
"pesce rosso"
"grande squalo bianco"
"Squalo tigre"
"Squalo martello"
"raggio elettrico"
"pastinaca"
"gallo"
"gallina"
"struzzo"
"rovo"
"cardellino"
"fringuello di casa"
"giunco"
"Zigolo indaco"
"pettirosso americano"
"bulbul"
"ghiandaia"
"gazza"
"cincillà"
"merlo acquaiolo americano"
"Nibbio (uccello rapace)"
"aquila calva"
"avvoltoio"
"Gufo grigio maggiore"
"Salamandra pezzata"
"tritone liscio"
"tritone"
"salamandra pezzata"
"axolotl"
"rana toro americana"
"raganella"
"Rana codina"
"Tartaruga di mare loggerhead"
"tartaruga di cuoio"
"tartaruga fangosa"
"tartaruga"
"tartaruga scatola"
"geco fasciato"
"iguana verde"
"Anolo della Carolina"
"lucertola a coda di rondine del deserto"
"agama"
"lucertola dal collo a frange"
"lucertola alligatore"
"Mostro di Gila"
"lucertola verde europea"
"camaleonte"
"Drago di Komodo"
"coccodrillo del Nilo"
"Alligatore americano"
"triceratopo"
"serpente verme"
"serpente dal collo ad anelli"
"serpente dal naso di maiale orientale"
"serpente verde liscio"
"serpente reale"
"serpente giarrettiera"
"serpente d'acqua"
"serpente delle vigne"
"serpente notturno"
"boa constrictor"
"Pitone africano delle rocce"
"cobra indiano"
"mamba verde"
"serpente di mare"
"vipera cornuta del Sahara"
"serpente a sonagli orientale"
"serpente a sonagli laterale"
"trilobite"
"Uomo del raccolto"
"scorpione"
"ragno giallo del giardino"
"ragno del granaio"
"Ragno europeo del giardino"
"vedova nera del sud"
"tarantola"
"ragno lupo"
"zecca"
"millepiedi"
"fagiano di monte"
"pernice bianca"
"gallo cedrone"
"fagiano di monte"
"pavone"
"quaglia"
"pernice"
"pappagallo grigio africano"
"ara"
"cacatua dalla cresta sulfurea"
"lorichetto"
"coucal"
"mangiatore di api"
"bucerotide"
"colibrì"
"jacamar"
"tucano"
"anatra"
"merganser dal petto rosso"
"oca"
"cigno nero"
"tusker"
"echidna"
"ornitorinco"
"wallaby"
"koala"
"vombato"
"medusa"
"anemone di mare"
"cervello di corallo"
"verme piatto"
"nematode"
"conchiglia"
"lumaca"
"lumaca"
"lumaca di mare"
"chitone"
"Nautilus a camera"
"Granchio di Dungeness"
"granchio di roccia"
"granchio violinista"
"granchio reale rosso"
"aragosta americana"
"aragosta spinosa"
"gambero di fiume"
"granchio eremita"
"isopode"
"cicogna bianca"
"cicogna nera"
"spatola"
"fenicottero"
"airone blu piccolo"
"airone bianco"
"Tarabuso"
"uccello gru"
"mignatta"
"Gallinelle comuni"
"Folaga americana"
"otarda"
"voltafaccia rubicondo"
"dunlin"
"pettegola comune"
"fraticello"
"beccaccia di mare"
"Pellicano"
"Pinguino reale"
"albatros"
"balena grigia"
"balena assassina"
"dugongo"
"leone marino"
"Chihuahua"
"Mento giapponese"
"Maltese"
"Pechinese"
"Shih Tzu"
"King Charles Spaniel"
"Papillon"
"Terrier giocattolo"
"Rhodesian Ridgeback"
"Levriero afgano"
"Basset Hound"
"Beagle"
"Bloodhound"
"Bluetick Coonhound"
"Black and Tan Coonhound"
"Treeing Walker Coonhound"
"Foxhound inglese"
"Redbone Coonhound"
"borzoi"
"Irish Wolfhound"
"Levriero italiano"
"Whippet"
"Segugio Ibiziano"
"Levriero norvegese"
"Otterhound"
"Saluki"
"Deerhound scozzese"
"Weimaraner"
"Staffordshire Bull Terrier"
"American Staffordshire Terrier"
"Bedlington Terrier"
"Border Terrier"
"Kerry Blue Terrier"
"Terrier Irlandese"
"Norfolk Terrier"
"Norwich Terrier"
"Yorkshire Terrier"
"Wire Fox Terrier"
"Lakeland Terrier"
"Sealyham Terrier"
"Airedale Terrier"
"Cairn Terrier"
"Terrier Australiano"
"Dandie Dinmont Terrier"
"Boston Terrier"
"Schnauzer in miniatura"
"Schnauzer Gigante"
"Schnauzer Standard"
"Terrier Scozzese"
"Tibetan Terrier"
"Australian Silky Terrier"
"Terrier Wheaten a pelo morbido"
"West Highland White Terrier"
"Lhasa Apso"
"Flat-Coated Retriever"
"Curly-coated Retriever"
"Golden Retriever"
"Labrador Retriever"
"Chesapeake Bay Retriever"
"Pointer tedesco a pelo corto"
"Vizsla"
"Setter inglese"
"Setter irlandese"
"Gordon Setter"
"Cane della Bretagna"
"Clumber Spaniel"
"Springer Spaniel inglese"
"Springer Spaniel gallese"
"Cocker Spaniel"
"Sussex Spaniel"
"Irish Water Spaniel"
"Kuvasz"
"Schipperke"
"Cane di Groenendael"
"Malinois"
"Briard"
"Kelpie australiano"
"Komondor"
"Vecchio cane da pastore inglese"
"Cane da pastore delle Shetland"
"Collie"
"Border Collie"
"Cane Bouvier des Flandres"
"Rottweiler"
"Cane da pastore tedesco"
"Dobermann"
"Pinscher in miniatura"
"Cane da montagna svizzero maggiore"
"Cane da montagna bernese"
"Appenzeller Sennenhund"
"Entlebucher Sennenhund"
"Boxer"
"Bullmastiff"
"Mastino tibetano"
"Bulldog francese"
"Alano"
"San Bernardo"
"husky"
"Alaskan Malamute"
"Husky siberiano"
"Dalmata"
"Affenpinscher"
"Basenji"
"Carlino"
"Leonberger"
"Cane di Terranova"
"Cane dei Grandi Pirenei"
"Samoiedo"
"Pomerania"
"Chow Chow"
"Keeshond"
"Griffone di Bruxelles"
"Pembroke Welsh Corgi"
"Cardigan Welsh Corgi"
"Barboncino giocattolo"
"Barboncino in miniatura"
"Barboncino standard"
"Cane messicano senza pelo (xoloitzcuintli)"
"lupo grigio"
"Lupo della tundra dell'Alaska"
"lupo rosso o lupo dalla criniera"
"coyote"
"dingo"
"dhole"
"Cane selvatico africano"
"iena"
"volpe rossa"
"volpe rossa"
"Volpe artica"
"volpe grigia"
"gatto soriano"
"gatto tigrato"
"Gatto persiano"
"Gatto siamese"
"Mau egiziano"
"coguaro"
"lince"
"leopardo"
"leopardo delle nevi"
"giaguaro"
"leone"
"tigre"
"ghepardo"
"orso bruno"
"Orso nero americano"
"orso polare"
"orso bradipo"
"mangusta"
"suricato"
"scarabeo tigre"
"coccinella"
"scarabeo di terra"
"coleottero delle corna"
"scarabeo delle foglie"
"scarabeo stercorario"
"scarabeo rinoceronte"
"tonchio"
"mosca"
"ape"
"formica"
"cavalletta"
"insetto grillo"
"insetto stecco"
"scarafaggio"
"mantide religiosa"
"cicala"
"cicala di mare"
"pizzo"
"libellula"
"damigella d'acqua"
"farfalla ammiraglio rosso"
"Farfalla ad anelli"
"farfalla monarca"
"piccola farfalla bianca"
"farfalla zolfo"
"farfalla dalle ali di gossamer"
"stella marina"
"riccio di mare"
"Cetriolo di mare"
"Coniglio dalla coda di cotone"
"lepre"
"Coniglio d'angora"
"criceto"
"Istrice"
"Scoiattolo volpe"
"marmotta"
"castoro"
"Porcellino d'India"
"cavallo sauro comune"
"zebra"
"Maiale"
"Cinghiale"
"facocero"
"ippopotamo"
"bue"
"bufalo d'acqua"
"bisonte"
"ariete (pecora maschio adulto)"
"pecora bighorn"
"Stambecco alpino"
"alcefalo"
"impala (antilope)"
"gazzella"
"Cammello arabo"
"lama"
"donnola"
"visone"
"Puzzola europea"
"furetto dai piedi neri"
"lontra"
"puzzola"
"tasso"
"armadillo"
"Bradipo a tre dita"
"orango"
"gorilla"
"scimpanzé"
"gibbone"
"siamang"
"cercopiteco"
"scimmia patas"
"babbuino"
"macaco"
"langur"
"colobo bianco e nero"
"scimmia proboscide"
"uistitì"
"cappuccino dalla testa bianca"
"scimmia urlatrice"
"scimmia titi"
"Scimmia ragno di Geoffroy"
"scimmia scoiattolo comune"
"lemure dalla coda ad anelli"
"indri"
"Elefante asiatico"
"Elefante africano della savana"
"panda rosso"
"panda gigante"
"pesce snoek"
"anguilla"
"salmone argentato"
"pesce bellezza di roccia"
"pesce pagliaccio"
"storione"
"pesce aguglia"
"pesce leone"
"pesce palla"
"abaco"
"abaya"
"abito accademico"
"fisarmonica"
"chitarra acustica"
"Portaerei"
"aereo di linea"
"dirigibile"
"altare"
"ambulanza"
"veicolo anfibio"
"orologio analogico"
"apiario"
"grembiule"
"bidone della spazzatura"
"fucile d'assalto"
"zaino"
"panificio"
"Trave di equilibrio"
"Palloncino"
"penna a sfera"
"Cerotto"
"banjo"
"balaustra / corrimano"
"bilanciere"
"barber chair"
"barbershop"
"fienile"
"barometro"
"barile"
"carriola"
"baseball"
"basket"
"bassetto"
"fagotto"
"cuffia"
"telo da bagno"
"vasca da bagno"
"station wagon"
"faro"
"bicchiere"
"cappello militare (pelle d'orso o shako)"
"bottiglia di birra"
"bicchiere di birra"
"campanile"
"bavaglino per bambini"
"bicicletta tandem"
"bikini"
"raccoglitore ad anelli"
"binocolo"
"casetta per uccelli"
"rimessa per barche"
"bob"
"cravatta bolo"
"poke bonnet"
"libreria"
"libreria"
"tappo di bottiglia"
"arco da caccia"
"papillon"
"targa commemorativa in ottone"
"reggiseno"
"frangiflutti"
"pettorale"
"scopa"
"secchio"
"fibbia"
"giubbotto antiproiettile"
"treno ad alta velocità"
"macelleria"
"taxi"
"calderone"
"candela"
"cannone"
"canoa"
"apriscatole"
"cardigan"
"auto specchio"
"carosello"
"kit di attrezzi"
"scatola di cartone / cartone"
"ruota dell'auto"
"sportello automatico"
"cassetta"
"lettore di cassette"
"castello"
"catamarano"
"CD player"
"violoncello"
"telefono cellulare"
"catena"
"chain-link fence"
"chain mail"
"motosega"
"cassapanca" 
"chiffonier"
"campana o campana a vento"
"armadio delle porcellane"
"Calza di Natale"
"chiesa"
"cinema"
"mannaia"
"dimora sulla scogliera"
"mantello"
"zoccoli"
"shaker per cocktail"
"tazza da caffè"
"coffeemaker"
"spirale o bobina"
"serratura a combinazione"
"tastiera del computer"
"negozio di caramelle"
"nave container"
"convertibile"
"cavatappi"
"cornetto"
"cowboy boot"
"cowboy hat"
"culla"
"gru da costruzione"
"casco di protezione"
"cassa"
"letto per bambini"
"Crock Pot"
"palla da croquet"
"Stampella"
"corazza"
"diga"
"scrivania"
"desktop computer"
"telefono a disco"
"pannolino"
"orologio digitale"
"orologio digitale"
"tavolo da pranzo"
"strofinaccio"
"lavastoviglie"
"freno a disco"
"bacino"
"slitta per cani"
"cupola"
"zerbino"
"impianto di perforazione"
"tamburo"
"bacchetta"
"manubrio"
"forno olandese"
"ventilatore elettrico"
"chitarra elettrica"
"locomotiva elettrica"
"centro d'intrattenimento"
"busta"
"macchina per caffè espresso"
"polvere per il viso"
"boa di piume"
"schedario"
"barca dei pompieri"
"camion dei pompieri"
"paravento"
"pennone"
"flauto"
"sedia pieghevole"
"casco da calcio"
"carrello elevatore"
"fontana"
"penna stilografica"
"letto a baldacchino"
"vagone merci"
"corno francese"
"padella"
"cappotto di pelliccia"
"camion della spazzatura"
"Maschera antigas o respiratore"
"pompa di benzina"
"calice"
"go-kart"
"pallina da golf"
"golf cart"
"gondola"
"gong"
"camice"
"grand piano"
"serra"
"radiatore a griglia"
"negozio di alimentari"
"ghigliottina"
"molletta per capelli"
"lacca per capelli"
"mezzo binario"
"martello"
"cestino"
"asciugacapelli"
"computer portatile"
"fazzoletto"
"hard disk drive"
"armonica"
"arpa"
"mietitrebbia"
"ascia di guerra"
"fondina"
"home theater"
"honeycomb"
"gancio"
"gonna a cerchio"
"barra orizzontale da ginnastica"
"veicolo trainato da cavalli"
"clessidra"
"iPod"
"ferro da stiro"
"zucca intagliata"
"jeans"
"jeep"
"maglietta"
"puzzle"
"Risciò"
"joystick"
"kimono"
"ginocchiera"
"nodo"
"camice da laboratorio"
"mestolo"
"paralume"
"computer portatile"
"tosaerba"
"tappo della lente"
"tagliacarte"
"biblioteca"
"scialuppa di salvataggio"
"accendino"
"limousine"
"transatlantico"
"Rossetto"
"scarpa da ginnastica"
"lozione"
"altoparlante per musica"
"lente d'ingrandimento"
"segheria"
"bussola magnetica"
"borsa del messaggero"
"cassetta della posta"
"collant"
"costume da bagno intero"
"tombino"
"maraca"
"marimba"
"maschera"
"fiammifero"
"maypole"
"labirinto"
"misurino"
"armadietto dei medicinali"
"megalite"
"microfono"
"forno a microonde"
"uniforme militare"
"barattolo del latte"
"minibus"
"minigonna"
"minivan"
"missile"
"guanto"
"ciotola di miscelazione"
"casa mobile"
"ford model t"
"modem"
"monastero"
"monitor"
"motorino"
"mortaio e pestello"
"graduation cap"
"moschea"
"zanzariera"
"vespa"
"mountain bike"
"tenda"
"mouse per computer"
"trappola per topi"
"furgone in movimento"
"museruola"
"chiodo di metallo"
"collare"
"collana"
"Ciuccio per bambini"
"computer portatile"
"obelisco"
"oboe"
"ocarina"
"contachilometri"
"filtro dell'olio"
"organo a canne"
"oscilloscopio"
"sopragonna"
"carro di buoi"
"maschera d'ossigeno"
"pacchetto / imballaggio del prodotto"
"paletta"
"ruota a pale"
"Lucchetto"
"pennello"
"pigiama"
"palazzo"
"flauto di pan"
"tovagliolo di carta"
"paracadute"
"barre parallele"
"panchina"
"parchimetro"
"vagone ferroviario"
"patio"
"telefono pubblico"
"piedistallo"
"astuccio per matite"
"temperamatite"
"profumo"
"Piastra di Petri"
"fotocopiatrice"
"plettro"
"Pickelhaube"
"recinzione"
"pickup"
"molo"
"salvadanaio"
"Bottiglia di pillole"
"Cuscino"
"palla da ping-pong"
"girandola"
"nave dei pirati"
"brocca per bevande"
"piano a blocchi"
"planetario"
"borsa di plastica"
"portapiatti"
"aratro agricolo"
"stantuffo"
"Macchina fotografica Polaroid"
"palo"
"Furgone della polizia"
"Poncho"
"tavolo da biliardo"
"Bottiglia di soda"
"Vaso per piante"
"ruota da vasaio"
"trapano elettrico"
"tappeto da preghiera"
"stampante"
"prigione"
"missile"
"proiettore"
"disco da hockey"
"sacco da boxe"
"borsa"
"penna d'oca"
"trapunta"
"auto da corsa"
"racchetta"
"radiatore"
"radio"
"radiotelescopio"
"barile per la pioggia"
"veicolo ricreativo"
"Mulinello da pesca"
"macchina fotografica reflex"
"frigorifero"
"telecomando"
"ristorante"
"revolver"
"fucile"
"sedia a dondolo"
"girarrosto"
"gomma da cancellare"
"palla da rugby"
"righello, bastone per misurare"
"scarpa da ginnastica"
"cassaforte"
"spilla da balia"
"saliera"
"sandalo"
"sarong"
"sassofono"
"Fodero"
"bilancia"
"scuolabus"
"goletta"
"tabellone segnapunti"
"Monitor CRT"
"Vite"
"cacciavite"
"cintura di sicurezza"
"macchina da cucire"
"scudo"
"negozio di scarpe"
"schermo shoji / divisorio per stanze"
"cestino della spesa"
"carrello della spesa"
"pala"
"cuffia da doccia"
"Tenda da doccia"
"sci"
"passamontagna"
"sacco a pelo"
"regolo calcolatore"
"porta scorrevole"
"slot machine"
"boccaglio"
"motoslitta"
"spazzaneve"
"distributore di sapone"
"pallone da calcio"
"calzino"
"collettore solare termico"
"sombrero"
"ciotola per zuppa"
"barra spaziatrice della tastiera"
"riscaldatore spaziale"
"navetta spaziale"
"spatola"
"motoscafo"
"ragnatela"
"fuso"
"auto sportiva"
"riflettore"
"palcoscenico"
"locomotiva a vapore"
"ponte ad arco passante"
"tamburo in acciaio"
"stetoscopio"
"sciarpa"
"muro di pietra"
"cronometro"
"stufa"
"colino"
"tram"
"barella"
"divano"
"stupa"
"sottomarino"
"tuta"
"meridiana"
"sunglasses (occhiali da sole)"
"occhiali da sole"
"crema solare"
"ponte sospeso"
"mocio"
"felpa"
"costume da bagno / pantaloncini"
"Altalena"
"interruttore elettrico"
"siringa"
"lampada da tavolo"
"Serbatoio"
"mangianastri"
"teiera"
"orsacchiotto"
"televisione"
"palla da tennis"
"tetto di paglia"
"tenda frontale"
"ditale"
"macchina per la trebbiatura"
"trono"
"tetto di tegole"
"tostapane"
"tabaccheria"
"tavoletta del water"
"torcia"
"totem"
"carro attrezzi"
"negozio di giocattoli"
"trattore"
"camion semirimorchio"
"vassoio"
"trench coat"
"triciclo"
"trimarano"
"treppiede"
"arco di trionfo"
"trolleybus"
"trombone"
"vasca idromassaggio"
"tornello"
"tastiera della macchina da scrivere"
"ombrello"
"monociclo"
"pianoforte verticale"
"aspirapolvere"
"vaso"
"soffitto a volta o ad arco"
"tessuto di velluto"
"distributore automatico"
"paramento"
"viadotto"
"violino"
"pallavolo"
"Ferro da stiro"
"orologio da parete"
"portafoglio"
"armadio"
"aereo militare"
"Lavello"
"lavatrice"
"Bottiglia d'acqua"
"brocca d'acqua"
"torre dell'acqua"
"brocca di whisky"
"Fischietto"
"parrucca per capelli"
"schermo della finestra"
"Tenda della finestra"
"cravatta Windsor"
"Bottiglia di vino"
"ala d'aereo"
"wok"
"cucchiaio di legno"
"Lana"
"Recinto con travi di legno"
"naufragio"
"barca a vela"
"yurta"
"sito web"
"fumetto"
"cruciverba"
"segnale stradale o di traffico"
"semaforo"
"giacca di polvere"
"menu"
"piatto"
"guacamole"
"consomme'"
"hot pot"
"trifle"
"gelato"
"ghiacciolo"
"baguette"
"bagel"
"pretzel"
"cheeseburger"
"hot dog"
"purè di patate"
"cavolo"
"broccoli"
"cavolfiore"
"zucchina"
"spaghetti alla zucca"
"zucca a ghianda"
"zucca butternut"
"cetriolo"
"carciofo"
"peperone"
"cardo"
"Fungo"
"Mela Granny Smith"
"fragola"
"arancia"
"limone"
"Fico"
"ananas"
"banana"
"jackfruit"
"anona (mela custard)"
"melograno"
"fieno"
"carbonara"
"sciroppo di cioccolato"
"pasta"
"polpettone"
"pizza"
"torta salata"
"burrito"
"vino rosso"
"espresso"
"tazza di tè"
"zabaione"
"montagna"
"bolla"
"scogliera"
"barriera corallina"
"geyser"
"riva del lago"
"promontorio"
"barra di sabbia"
"spiaggia"
"valle"
"vulcano"
"giocatore di baseball"
"sposo"
"subacqueo"
"colza"
"margherita"
"pantofola gialla della signora"
"mais"
"ghianda"
"rosa canina"
"Seme di ippocastano"
"fungo del corallo"
"agarico"
"gyromitra"
"fungo del corno di rovo"
"fungo stella di terra"
"fungo gallina dei boschi"
"boleto"
"pannocchia di mais"
"carta igienica"]'''.replace("\'", "'").replace('" "', '"').replace('""', '"').replace('\n', ', ')   #.replace('"\n', '", ').replace('\n', '", ')

In [ ]:
["tinca", "pesce rosso", "grande squalo bianco", "Squalo tigre", "Squalo martello", "raggio elettrico", "pastinaca", "gallo", "gallina", "struzzo", "rovo", "cardellino", "fringuello di casa", "giunco", "Zigolo indaco", "pettirosso americano", "bulbul", "ghiandaia, "gazza, "cincillà, "merlo acquaiolo americano", "Nibbio (uccello rapace), "aquila calva, "avvoltoio, "Gufo grigio maggiore, "Salamandra pezzata, "tritone liscio", "tritone", "salamandra pezzata, "axolotl, "rana toro americana, "raganella", "Rana codina", "Tartaruga di mare loggerhead", "tartaruga di cuoio", "tartaruga fangosa", "tartaruga", "tartaruga scatola", "geco fasciato", "iguana verde", "Anolo della Carolina", "lucertola a coda di rondine del deserto", "agama", "lucertola dal collo a frange", "lucertola alligatore", "Mostro di Gila", "lucertola verde europea, "camaleonte", "Drago di Komodo", "coccodrillo del Nilo", "Alligatore americano", "triceratopo", "serpente verme", "serpente dal collo ad anelli", "serpente dal naso di maiale orientale", "serpente verde liscio", "serpente reale", "serpente giarrettiera", "serpente d\'acqua", "serpente delle vigne", "serpente notturno", "boa constrictor", "Pitone africano delle rocce", "cobra indiano", "mamba verde", "serpente di mare", "vipera cornuta del Sahara", "serpente a sonagli orientale", "serpente a sonagli laterale", "trilobite", "Uomo del raccolto", "scorpione", "ragno giallo del giardino", "ragno del granaio", "Ragno europeo del giardino", "vedova nera del sud", "tarantola", "ragno lupo", "zecca", "millepiedi", "fagiano di monte", "pernice bianca, "gallo cedrone", "fagiano di monte", "pavone", "quaglia", "pernice", "pappagallo grigio africano", "ara", "cacatua dalla cresta sulfurea", "lorichetto", "coucal", "mangiatore di api", "bucerotide", "colibrì", "jacamar", "tucano", "anatra", "merganser dal petto rosso, "oca, "cigno nero, "tusker", "echidna", "ornitorinco, "wallaby", "koala", "vombato", "medusa", "anemone di mare", "cervello di corallo", "verme piatto", "nematode", "conchiglia", "lumaca", "lumaca", "lumaca di mare", "chitone", "Nautilus a camera", "Granchio di Dungeness", "granchio di roccia", "granchio violinista", "granchio reale rosso", "aragosta americana", "aragosta spinosa", "gambero di fiume", "granchio eremita", "isopode", "cicogna bianca", "cicogna nera", "spatola", "fenicottero", "airone blu piccolo", "airone bianco", "Tarabuso, "uccello gru", "mignatta", "Gallinelle comuni", "Folaga americana", "otarda", "voltafaccia rubicondo", "dunlin", "pettegola comune", "fraticello", "beccaccia di mare", "Pellicano", "Pinguino reale", "albatros", "balena grigia", "balena assassina", "dugongo", "leone marino", "Chihuahua", "Mento giapponese", "Maltese", "Pechinese", "Shih Tzu", "King Charles Spaniel", "Papillon", "Terrier giocattolo", "Rhodesian Ridgeback", "Levriero afgano", "Basset Hound", "Beagle", "Bloodhound", "Bluetick Coonhound", "Black and Tan Coonhound", "Treeing Walker Coonhound", "Foxhound inglese", "Redbone Coonhound", "borzoi", "Irish Wolfhound", "Levriero italiano", "Whippet", "Segugio Ibiziano", "Levriero norvegese", "Otterhound", "Saluki", "Deerhound scozzese", "Weimaraner", "Staffordshire Bull Terrier", "American Staffordshire Terrier", "Bedlington Terrier", "Border Terrier", "Kerry Blue Terrier", "Terrier Irlandese", "Norfolk Terrier", "Norwich Terrier", "Yorkshire Terrier", "Wire Fox Terrier", "Lakeland Terrier", "Sealyham Terrier", "Airedale Terrier", "Cairn Terrier", "Terrier Australiano", "Dandie Dinmont Terrier", "Boston Terrier", "Schnauzer in miniatura", "Schnauzer Gigante", "Schnauzer Standard", "Terrier Scozzese", "Tibetan Terrier", "Australian Silky Terrier", "Terrier Wheaten a pelo morbido", "West Highland White Terrier", "Lhasa Apso", "Flat-Coated Retriever", "Curly-coated Retriever", "Golden Retriever", "Labrador Retriever", "Chesapeake Bay Retriever", "Pointer tedesco a pelo corto", "Vizsla", "Setter inglese", "Setter irlandese", "Gordon Setter", "Cane della Bretagna", "Clumber Spaniel", "Springer Spaniel inglese", "Springer Spaniel gallese", "Cocker Spaniel", "Sussex Spaniel", "Irish Water Spaniel", "Kuvasz", "Schipperke", "Cane di Groenendael", "Malinois", "Briard", "Kelpie australiano", "Komondor", "Vecchio cane da pastore inglese", "Cane da pastore delle Shetland", "Collie", "Border Collie", "Cane Bouvier des Flandres", "Rottweiler", "Cane da pastore tedesco", "Dobermann", "Pinscher in miniatura", "Cane da montagna svizzero maggiore", "Cane da montagna bernese", "Appenzeller Sennenhund", "Entlebucher Sennenhund", "Boxer", "Bullmastiff", "Mastino tibetano", "Bulldog francese", "Alano", "San Bernardo", "husky", "Alaskan Malamute", "Husky siberiano", "Dalmata", "Affenpinscher", "Basenji", "Carlino", "Leonberger", "Cane di Terranova", "Cane dei Grandi Pirenei", "Samoiedo", "Pomerania", "Chow Chow", "Keeshond", "Griffone di Bruxelles", "Pembroke Welsh Corgi", "Cardigan Welsh Corgi", "Barboncino giocattolo", "Barboncino in miniatura", "Barboncino standard", "Cane messicano senza pelo (xoloitzcuintli)", "lupo grigio", "Lupo della tundra dell\'Alaska", "lupo rosso o lupo dalla criniera", "coyote", "dingo", "dhole", "Cane selvatico africano", "iena", "volpe rossa", "volpe rossa", "Volpe artica", "volpe grigia", "gatto soriano", "gatto tigrato", "Gatto persiano", "Gatto siamese", "Mau egiziano", "coguaro", "lince", "leopardo", "leopardo delle nevi", "giaguaro", "leone", "tigre", "ghepardo", "orso bruno", "Orso nero americano", "orso polare", "orso bradipo", "mangusta", "suricato", "scarabeo tigre", "coccinella", "scarabeo di terra", "coleottero delle corna", "scarabeo delle foglie", "scarabeo stercorario", "scarabeo rinoceronte", "tonchio", "mosca", "ape", "formica", "cavalletta", "insetto grillo", "insetto stecco", "scarafaggio", "mantide religiosa", "cicala", "cicala di mare", "pizzo", "libellula", "damigella d'acqua", "farfalla ammiraglio rosso", "Farfalla ad anelli", "farfalla monarca", "piccola farfalla bianca", "farfalla zolfo", "farfalla dalle ali di gossamer", "stella marina", "riccio di mare", "Cetriolo di mare", "Coniglio dalla coda di cotone", "lepre", "Coniglio d'angora", "criceto", "Istrice", "Scoiattolo volpe", "marmotta", "castoro", "Porcellino d\'India", "cavallo sauro comune", "zebra", "Maiale", "Cinghiale", "facocero", "ippopotamo", "bue", "bufalo d'acqua", "bisonte", "ariete", "pecora bighorn", "Stambecco alpino", "alcefalo", "impala", "gazzella", "Cammello arabo", "lama", "donnola", "visone", "Puzzola europea", "furetto dai piedi neri", "lontra", "puzzola", "tasso", "armadillo", "Bradipo a tre dita", "orango", "gorilla", "scimpanzé", "gibbone", "siamang", "cercopiteco", "scimmia patas", "babbuino", "macaco", "langur", "colobo bianco e nero", "scimmia proboscide", "uistitì", "cappuccino dalla testa bianca", "scimmia urlatrice", "scimmia titi", "Scimmia ragno di Geoffroy", "scimmia scoiattolo comune", "lemure dalla coda ad anelli", "indri", "Elefante asiatico", "Elefante africano della savana", "panda rosso", "panda gigante", "pesce snoek", "anguilla", "salmone argentato", "pesce bellezza di roccia", "pesce pagliaccio", "storione", "pesce aguglia", "pesce leone", "pesce palla", "abaco", "abaya", "abito accademico", "fisarmonica", "chitarra acustica", "Portaerei", "aereo di linea", "dirigibile", "altare", "ambulanza", "veicolo anfibio", "orologio analogico", "apiario", "grembiule", "bidone della spazzatura", "fucile d'assalto", "zaino", "panificio", "Trave di equilibrio", "Palloncino", "penna a sfera", "Cerotto", "banjo", "balaustra", "bilanciere", "barber chair", "barbiere", "fienile", "barometro", "barile", "carriola", "baseball", "basket", "bassetto", "fagotto", "cuffia", "telo da bagno", "vasca da bagno", "station wagon", "faro", "bicchiere", "cappello pelle d'orso", "bottiglia di birra", "bicchiere di birra", "campanile", "bavaglino per bambini", "bicicletta tandem", "bikini", "raccoglitore ad anelli", "binocolo", "casetta per uccelli", "rimessa per barche", "bob", "cravatta bolo", "poke bonnet", "libreria", "libreria", "tappo di bottiglia", "arco da caccia", "papillon", "targa commemorativa in ottone", "reggiseno", "frangiflutti", "pettorale", "scopa", "secchio", "fibbia", "giubbotto antiproiettile", "treno ad alta velocità", "macelleria", "taxi", "calderone", "candela", "cannone", "canoa", "apriscatole", "cardigan", "auto specchio", "carosello", "kit di attrezzi", "scatola di cartone / cartone", "ruota dell\'auto", "sportello automatico", "cassetta", "lettore di cassette", "castello", "catamarano", "CD player", "violoncello", "telefono cellulare", "catena", "recinzione a catena", "cotta di maglia", "motosega", "cassapanca" , "chiffonier", "campana o campana a vento", "armadio delle porcellane", "Calza di Natale", "chiesa", "cinema", "mannaia", "dimora sulla scogliera", "mantello", "zoccoli", "shaker per cocktail", "tazza da caffè", "coffeemaker", "spirale o bobina", "serratura a combinazione", "tastiera del computer", "negozio di caramelle", "nave container", "convertibile", "cavatappi", "cornetto", "cowboy boot", "cowboy hat", "culla", "gru da costruzione", "casco di protezione", "cassa", "letto per bambini", "Crock Pot", "palla da croquet", "Stampella", "corazza", "diga", "scrivania", "desktop computer", "telefono a disco", "pannolino", "orologio digitale", "orologio digitale", "tavolo da pranzo", "strofinaccio", "lavastoviglie", "freno a disco", "bacino", "slitta per cani", "cupola", "zerbino", "impianto di perforazione", "tamburo", "bacchetta", "manubrio", "forno olandese", "ventilatore elettrico", "chitarra elettrica", "locomotiva elettrica", "centro d\'intrattenimento", "busta", "macchina per caffè espresso", "polvere per il viso", "boa di piume", "schedario", "barca dei pompieri", "camion dei pompieri", "paravento", "pennone", "flauto", "sedia pieghevole", "casco da calcio", "carrello elevatore", "fontana", "penna stilografica", "letto a baldacchino", "vagone merci", "corno francese", "padella", "cappotto di pelliccia", "camion della spazzatura", "Maschera antigas o respiratore", "pompa di benzina", "calice", "go-kart", "pallina da golf", "golf cart", "gondola", "gong", "camice", "grand piano", "serra", "radiatore a griglia", "negozio di alimentari", "ghigliottina", "molletta per capelli", "lacca per capelli", "mezzo binario", "martello", "cestino", "asciugacapelli", "computer portatile", "fazzoletto", "hard disk drive", "armonica", "arpa", "mietitrebbia", "ascia di guerra", "fondina", "home theater", "honeycomb", "gancio", "gonna a cerchio", "barra orizzontale da ginnastica", "veicolo trainato da cavalli", "clessidra", "iPod", "ferro da stiro", "zucca intagliata", "jeans", "jeep", "maglietta", "puzzle", "Risciò", "joystick", "kimono", "ginocchiera", "nodo", "camice da laboratorio", "mestolo", "paralume", "computer portatile", "tosaerba", "tappo della lente", "tagliacarte", "biblioteca", "scialuppa di salvataggio", "accendino", "limousine", "transatlantico", "Rossetto", "scarpa da ginnastica", "lozione", "altoparlante per musica", "lente d\'ingrandimento", "segheria", "bussola magnetica", "borsa del messaggero", "cassetta della posta", "collant", "costume da bagno intero", "tombino", "maraca", "marimba", "maschera", "fiammifero", "maypole", "labirinto", "misurino", "armadietto dei medicinali", "megalite", "microfono", "forno a microonde", "uniforme militare", "barattolo del latte", "minibus", "minigonna", "minivan", "missile", "guanto", "ciotola di miscelazione", "casa mobile", "ford model t", "modem", "monastero", "monitor", "motorino", "mortaio e pestello", "graduation cap", "moschea", "zanzariera", "vespa", "mountain bike", "tenda", "mouse per computer", "trappola per topi", "furgone in movimento", "museruola", "chiodo di metallo", "collare", "collana", "Ciuccio per bambini", "computer portatile", "obelisco", "oboe", "ocarina", "contachilometri", "filtro dell\'olio", "organo a canne", "oscilloscopio", "sopragonna", "carro di buoi", "maschera d\'ossigeno", "pacchetto / imballaggio del prodotto", "paletta", "ruota a pale", "Lucchetto", "pennello", "pigiama", "palazzo", "flauto di pan", "tovagliolo di carta", "paracadute", "barre parallele", "panchina", "parchimetro", "vagone ferroviario", "patio", "telefono pubblico", "piedistallo", "astuccio per matite", "temperamatite", "profumo", "Piastra di Petri", "fotocopiatrice", "plettro", "Pickelhaube", "recinzione", "pickup", "molo", "salvadanaio", "Bottiglia di pillole", "Cuscino", "palla da ping-pong", "girandola", "nave dei pirati", "brocca per bevande", "piano a blocchi", "planetario", "borsa di plastica", "portapiatti", "aratro agricolo", "stantuffo", "Macchina fotografica Polaroid", "palo", "Furgone della polizia", "Poncho", "tavolo da biliardo", "Bottiglia di soda", "Vaso per piante", "ruota da vasaio", "trapano elettrico", "tappeto da preghiera", "stampante", "prigione", "missile", "proiettore", "disco da hockey", "sacco da boxe", "borsa", "penna d\'oca", "trapunta", "auto da corsa", "racchetta", "radiatore", "radio", "radiotelescopio", "barile per la pioggia", "veicolo ricreativo", "Mulinello da pesca", "macchina fotografica reflex", "frigorifero", "telecomando", "ristorante", "revolver", "fucile", "sedia a dondolo", "girarrosto", "gomma da cancellare", "palla da rugby", "righello, bastone per misurare", "scarpa da ginnastica", "cassaforte", "spilla da balia", "saliera", "sandalo", "sarong", "sassofono", "Fodero", "bilancia", "scuolabus", "goletta", "tabellone segnapunti", "Monitor CRT", "Vite", "screwdriver" (cacciavite), "cintura di sicurezza", "macchina da cucire", "scudo", "negozio di scarpe", "schermo shoji / divisorio per stanze", "cestino della spesa", "carrello della spesa", "pala", "cuffia da doccia", "Tenda da doccia, "sci", "passamontagna, "sacco a pelo", "regolo calcolatore", "porta scorrevole", "slot machine", "boccaglio", "motoslitta", "spazzaneve", "distributore di sapone", "pallone da calcio", "calzino", "collettore solare termico", "sombrero", "ciotola per zuppa", "barra spaziatrice della tastiera", "riscaldatore spaziale", "navetta spaziale", "spatola", "motoscafo", "ragnatela", "fuso", "auto sportiva", "riflettore", "palcoscenico", "locomotiva a vapore, "ponte ad arco passante", "tamburo in acciaio", "stetoscopio, "sciarpa", "muro di pietra", "cronometro", "stufa", "colino", "tram", "barella", "divano", "stupa", "sottomarino", "tuta", "meridiana", "sunglasses" (occhiali da sole), "occhiali da sole", "crema solare", "ponte sospeso", "mocio", "felpa", "costume da bagno / pantaloncini", "Altalena", "interruttore elettrico", "siringa", "lampada da tavolo", "Serbatoio", "mangianastri", "teiera", "orsacchiotto", "televisione", "palla da tennis", "tetto di paglia", "tenda frontale", "ditale", "macchina per la trebbiatura", "trono", "tetto di tegole", "tostapane", "tabaccheria", "tavoletta del water", "torcia", "totem", "carro attrezzi", "negozio di giocattoli", "trattore", "camion semirimorchio", "vassoio", "trench coat", "triciclo", "trimarano", "treppiede", "arco di trionfo", "trolleybus", "trombone", "vasca idromassaggio", "tornello", "tastiera della macchina da scrivere", "ombrello", "monociclo", "pianoforte verticale", "aspirapolvere", "vaso", "soffitto a volta o ad arco", "tessuto di velluto", "distributore automatico", "paramento", "viadotto", "violino", "pallavolo", "Ferro da stiro", "orologio da parete", "portafoglio", "armadio", "aereo militare", "Lavello", "lavatrice", "Bottiglia d\'acqua", "brocca d\'acqua", "torre dell\'acqua", "brocca di whisky", "Fischietto", "parrucca per capelli", "schermo della finestra", "Tenda della finestra", "cravatta Windsor", "Bottiglia di vino", "ala d\'aereo", "wok", "cucchiaio di legno", "Lana", "Recinto con travi di legno", "naufragio", "barca a vela", "yurta", "sito web", "fumetto", "cruciverba", "segnale stradale o di traffico", "semaforo", "giacca di polvere", "menu", "piatto", "guacamole", "consomme\'", "hot pot", "trifle", "gelato", "ghiacciolo", "baguette", "bagel", "pretzel", "cheeseburger", "hot dog", "purè di patate", "cavolo", "broccoli", "cavolfiore", "zucchina", "spaghetti alla zucca", "zucca a ghianda", "zucca butternut", "cetriolo", "carciofo", "peperone", "cardo", "Fungo", "Mela Granny Smith", "fragola", "arancia", "limone", "Fico", "ananas", "banana", "jackfruit", "anona (mela custard)", "melograno", "fieno", "carbonara, "sciroppo di cioccolato", "pasta", "polpettone", "pizza", "torta salata", "burrito", "vino rosso", "espresso", "tazza di tè", "zabaione", "montagna", "bolla", "scogliera", "barriera corallina", "geyser", "riva del lago", "promontorio", "barra di sabbia", "spiaggia", "valle", "vulcano", "giocatore di baseball", "sposo", "subacqueo", "colza", "margherita", "pantofola gialla della signora", "mais", "ghianda", "rosa canina", "Seme di ippocastano", "fungo del corallo", "agarico", "gyromitra", "fungo del corno di rovo", "fungo stella di terra", "fungo gallina dei boschi", "boleto", "pannocchia di mais", "carta igienica"]